In [27]:
import pandas as pd
import boto3
import json
import pprint as pp
import sys
from io import StringIO


s3 = boto3.client('s3')
sys.setrecursionlimit(4000)

bucket_name = 'data-402-final-project'
talent_prefix = 'Talent/'

### Read JSON file from local machine

In [28]:
# df = pd.read_json('talent.json')
# df

In [29]:

def list_all_objects(bucket, prefix):
    all_objects = []
    continuation_token = None

    while True:
        if continuation_token:
            response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix, ContinuationToken=continuation_token)
        else:
            response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
            #pp.pprint(response)

        if 'Contents' in response:
            all_objects.extend([obj['Key'] for obj in response['Contents']])

        if not response.get('NextContinuationToken'):
            break

        continuation_token = response['NextContinuationToken']

    return all_objects

In [30]:
def check_talent_files(bucket, prefix):
    talent_list = list()
    files = list_all_objects(bucket, prefix)
    if not files:
        print("No files found in the Talent folder.")
        return

    first_file_key = files[0]
    obj = s3.get_object(Bucket=bucket, Key=first_file_key)
    first_file_data = json.load(obj['Body'])
    expected_keys = set(first_file_data.keys())

    inconsistent_files = []
    missing_values_count = 0
    number_of_talent_files = 0
    number_of_json_files = 0
    print("Please wait...")
    for file_key in files:
        obj = s3.get_object(Bucket=bucket, Key=file_key)
        try:
            if file_key[-3:] == 'son':
                number_of_json_files += 1
                data = json.load(obj['Body'])

                if set(data.keys()) != expected_keys:
                    inconsistent_files.append(file_key)
                #print(data)
                talent_list.append(data)


                missing_values_count += sum(1 for k, v in data.items() if v in [None, "", [], {}])
        except Exception as e:
            print(f"Error reading file {file_key}: {e}")
        number_of_talent_files += 1

    # print("Talent Files Summary:")
    # print(f"Number of inconsistent files: {len(inconsistent_files)}")
    # print(f"Total number of missing values: {missing_values_count}")
    # print(f"Total file count: {number_of_talent_files}, with {number_of_json_files} json files.")
    return json.dumps(talent_list)

# check_talent_files(bucket_name, talent_prefix)

json_files = check_talent_files(bucket_name, talent_prefix)




Please wait...


In [32]:
df = pd.read_json(StringIO(json_files))
df

,name,date,tech_self_score,strengths,weaknesses,self_development,geo_flex,financial_support_self,result,course_interest
0,Stillmann Castano,2019-08-22,"{'C#': 6, 'Java': 5, 'R': 2, 'JavaScript': 2}",[Charisma],"[Distracted, Impulsive, Introverted]",Yes,Yes,Yes,Pass,Business
1,Hilary Willmore,2019-01-08,"{'Python': 1, 'C#': 4, 'Java': 2, 'C++': 4}","[Patient, Curious, Problem Solving]","[Overbearing, Chatty, Indifferent]",No,Yes,Yes,Fail,Data
2,Efrem Whipple,2019-08-22,"{'Ruby': 4, 'C++': 4}","[Courteous, Independent, Patient]","[Introverted, Impulsive, Anxious]",Yes,Yes,Yes,Pass,Business
3,Sydel Fenne,2019-08-28,"{'Java': 3, 'SPSS': 4}",[Passionate],"[Perfectionist, Sensitive]",Yes,Yes,Yes,Pass,Data
4,Michel Lebarree,2019-07-08,"{'Python': 3, 'Java': 4, 'Ruby': 1, 'R': 2, 'P...",[Versatile],"[Controlling, Perfectionist, Chatty]",Yes,Yes,Yes,Pass,Engineering
...,...,...,...,...,...,...,...,...,...,...
3100,Jacky Reilingen,2019-04-04,"{'C#': 2, 'Java': 6, 'R': 1, 'SPSS': 4}",[Versatile],"[Indifferent, Intolerant, Introverted]",Yes,No,Yes,Fail,Business
3101,Phillis Lyfield,2019-10-04,"{'C#': 4, 'Java': 4, 'Ruby': 4, 'PHP': 1}","[Organisation, Independent, Determined]","[Sensitive, Overbearing, Impatient]",Yes,Yes,Yes,Pass,Engineering
3102,Celle Barlas,2019-04-16,"{'R': 2, 'C++': 1, 'JavaScript': 4}",[Problem Solving],[Critical],Yes,Yes,Yes,Pass,Engineering
3103,Scott Duny,2019-11-04,{'Ruby': 3},"[Reliable, Perfectionism, Problem Solving]",[Stubborn],Yes,Yes,Yes,Pass,Data


In [ ]:
df.loc[0]

In [ ]:
df.isnull().sum()

In [ ]:
df[df['tech_self_score'].isnull()]

In [ ]:
df = df.fillna('{}')
df.loc[24]

In [ ]:
df['tech_self_score'] = df['tech_self_score'].apply(str)
tech_score_types = df['tech_self_score'].apply(type)
print(tech_score_types)

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated()

In [ ]:
df['weaknesses'] = df['weaknesses'].apply(str)
df['strengths'] = df['strengths'].apply(str)
df[df.duplicated() == True]

In [ ]:
df = df.drop_duplicates()
df